In [16]:
import mbuild as mb
import os
from parmed import load_file
import re
from mbuild.lib.atoms import H
import numpy as np
import random
import pubchempy as pcp
from rdkit import Chem 
from  Main_functions import *

The following example is based on the work carried out by Chehrazi 
https://pubs.acs.org/doi/10.1021/acsomega.3c10108

<img src="../images/TETA_TMC_image.png">


First step the main amine, TETA (triethylenetetramine) is fetched by name from the PubChem database.

In [17]:
_, cid = fetch_and_plug_CR("triethylenetetramine")
smiles, _ = get_mol_CR(cid)
Polymer_Compound = Build_3D_CR(smiles) 
Polymer_Compound.visualize()

Base name: triethylenetetramine, SMILES: None, CID: 5565
C(CNCCNCCN)N


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Now the monomer has to be modified by adding the ports and removing a hydrogen from the ends of the TETA chain

In [18]:
ports_to_add_Dummy_atoms_Polymer_Compound = {

     "NH2C" : [
        ["N", # The first atom is the "anchor" or base atom 
           "H", "H", "C"], # Bonded atoms to Carbon 
        ["H"], # Atom to remove and create port
        [3], # Total number of bonds in carbon atom 
        ],
}

In [19]:
compound_1_port_PC = Add_port_CR(Polymer_Compound,
                                type = "NH2C",
                                name = "up",
                                manual_ports= True,
                                ports_dict = ports_to_add_Dummy_atoms_Polymer_Compound)    
    
print("Added ports to the molecule")
tmp_cmp_PC = mb.clone(compound_1_port_PC)

#Adding second port to the repeat unit        
compound_with_2_ports_PC = Add_port_CR(tmp_cmp_PC,
                                    type = "NH2C",
                                    name = "down",
                                    manual_ports= True,
                                    ports_dict = ports_to_add_Dummy_atoms_Polymer_Compound)      
print("Added 2 ports to the molecule")



Added ports to the molecule


Added 2 ports to the molecule


In [20]:
compound_with_2_ports_PC.visualize(show_ports=True)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

After adding the ports to the polymer monomer, now the Trimesoyl Chloride (TMC) is loaded and the ports are added

In [21]:
_, cid = fetch_and_plug_CR("trimesoyl chloride")
smiles, _ = get_mol_CR(cid)
TMC = Build_3D_CR(smiles) 
TMC.visualize()
tmc_c = mb.clone(TMC)

Base name: trimesoyl chloride, SMILES: None, CID: 78138
C1=C(C=C(C=C1C(=O)Cl)C(=O)Cl)C(=O)Cl


In [22]:
ports_to_add_Dummy_atoms_TMC = {

     "C_Cl" : [
        ["C", # The first atom is the "anchor" or base atom 
        "C", "O","Cl"], # Bonded atoms to Carbon 
        ["Cl"], # Atom to remove and create port
        [3], # Total number of bonds in carbon atom 
        ],
}

In [23]:
compound_1_port_TMC = Add_port_CR(tmc_c,
                                type = "C_Cl",
                                name = "up",
                                manual_ports= True,
                                ports_dict = ports_to_add_Dummy_atoms_TMC)    
    
print("Added ports to the molecule")
tmp_cmp_TMC = mb.clone(compound_1_port_TMC)

#Adding second port to the repeat unit        
compound_with_2_ports_TMC = Add_port_CR(tmp_cmp_TMC,
                                    type = "C_Cl",
                                    name = "down",
                                    manual_ports= True,
                                    ports_dict = ports_to_add_Dummy_atoms_TMC)      
print("Added 2 ports to the molecule")







Added ports to the molecule






Added 2 ports to the molecule


In [24]:
compound_with_2_ports_TMC.visualize(show_ports = True)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Now the polymer is built with the combination of both molecules using mbuild's functionality for co-polymers

In [25]:
#repeat_units_AB_copolymer = 5

In [26]:
#COP_polymer = mb.recipes.Polymer(monomers=[compound_with_2_ports_PC, compound_with_2_ports_TMC ])
#COP_polymer.build( n=repeat_units_AB_copolymer, sequence="AB")
#COP_polymer.visualize()


Now that we have built the main polymer chain the crosslinking takes place, with TETA as the crosslinker

In [27]:
Single_Repeat_Unit = mb.recipes.Polymer(monomers=[compound_with_2_ports_PC, compound_with_2_ports_TMC ])
Single_Repeat_Unit.build( n=1, sequence="AB")
Single_Repeat_Unit.visualize()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [28]:

RU = 5
N_chains = 5
n_sites = 1

#Repeat_unit_from_file = mb.load("../files/Repeat_Unit_2.mol2")
dummy_atom_dictionary = {
                         "F": "H",}


box_system, Number_Crosslinkers_Used, Real_Degree_Crosslinking, Polymer_Residue_Name, Crosslinker_Resiude_Name = Crosslink_Pipeline(
#box_system = Crosslink_Pipeline(
    main_name = "EPON",                                   # Monomer to polimerize gives PVA ; not vinylic alcohol as it's insaturated
    Pre_existing_repeat_unit =True,
    Repeat_unit_compound_with_ports = compound_with_2_ports_DA, # Here we pass already the repeat unit with ports

    Dummy_atoms = True,
    Dummy_atom_dictionary = dummy_atom_dictionary,

    
    repeat_units = 5,                                       # Repeat Units in polymer chain
    number_of_polymers = 3,                                   # Number of polymer chains inside the box
    depth_value = 3.0,                                          # Spacing between polymer chains in box
    Polymer_residue_name = "POL",                             # name of polymer chain residue
    Polymer_port_name = "Port",                               # name of polymer chain backbone ports

                                                              #Crosslinker details
    crosslinker_name = "BASE",                                # Name of crosslinker agent
    Crosslinker_residue_name = "CRS",                         # name of crosslinker residue name
    Crosslinker_port_name = "Site",                           # name of cr chain backbone ports
    Crosslinker_compound = CR_COMPOUND,
    

 
                                                              #Crosslinking details
    sites_per_repeat_unit = 1,                                # Number of functional groups (f.ex hydroxyls in PVA in each chain)
    desired_CR_degree = 40,                                   # Degree in %, that percentage (f.ex number of hydroxyls attached to the crosslinker in PVA, not hydroxyls anymore)
    
    
    polymer_backbone_target_list = [
                            {
                            "anchor_atom" : "N",
                            "target_atom" : "F",
                            "bonded_atoms": ["C", "H", "F"], 
                            "anchor_atom_n_bonds" : 4,
                            },
                            
                          
                            ],
    
    
    crosslinker_backbone_target_list = [
                        {
                        "anchor_atom" : "C",
                        "target_atom" : ["S"],
                        "anchor_atom_n_bonds" : 4,
                        "extra_pruning": [],
                        "exclude": [],
                        },
                        
                    
    ],
    
    prune_extra_flag =False,                               # No pruning extra atoms 
    crosslink_sites_per_molecule = 1 ,                     # Sites per molecule  (forms 1 connection with each polymer chain) f.ex glutaraldehyde forms 2 per chain
        
   
    

        
    
    
)


for suffix in ["mol2", "pdb"]:
    box_system.save(f"output/5_3_EPON-862_DETDA-Short.{suffix}",overwrite = True)

NameError: name 'compound_with_2_ports_DA' is not defined